In [3]:
%matplotlib inline
import os
import time
import requests
import urllib3
from tqdm import tqdm
from lxml import etree
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import numpy as np
import seaborn as sn
import warnings
import itertools
warnings.filterwarnings('ignore')
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
year = "20172018"
for i in tqdm(range(1, 99)):
    for j in range(1, 99):
        file = "h-tk-%s-%s-%s" %(year, str(i), str(j))
        url = "https://zoek.officielebekendmakingen.nl/%s" %file
        resp = requests.get(url + ".xml", verify=False, allow_redirects=False)
        if resp.status_code == 200:
            hand_file = open('HandelingenTK/%s.xml' %file, 'wb')
            hand_file.write(resp.content)
            hand_file.close()
            resp = requests.get(url+"/metadata.xml", verify=False, \
                                                allow_redirects=False)
            meta_file = open('HandelingenTKmeta/meta%s.xml' %file, 'wb')
            meta_file.write(resp.content)
            meta_file.close()    
        else:
            break

 19%|███████████████▉                                                                  | 19/98 [12:40<52:41, 40.02s/it]

In [ ]:
enddict = {}
i = 0
for file in tqdm(os.listdir('HandelingenTK/')):
    context = etree.iterparse('HandelingenTK/' + file,  tag='spreekbeurt')
    for _, elem in context:
        surname = elem.findtext('spreker/naam/achternaam')
        party = elem.findtext('spreker/politiek')
        text = ' '.join([elem2.findtext('al') for elem2 \
                         in elem.findall('tekst/al-groep') \
                          if elem2.findtext('al') is not None])
        enddict[i] = [surname, party, text, file]
        i += 1
df = pd.DataFrame.from_dict(enddict, orient='index')
df = df.rename(columns={0:'achternaam', 1:'partij', 2:'tekst', 3:'file'})
df.to_csv